<a href="https://colab.research.google.com/github/denmj/Course_2_Math_for_ML/blob/master/TF_FNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np

In [0]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [0]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [52]:
n_inputs = 28*28 
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01
X_train.shape

(55000, 784)

In [0]:
reset_graph()

In [0]:
y = tf.placeholder(tf.int32, shape=(None), name='y')
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')


In [0]:
with tf.name_scope('FNN'):



    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden_1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden_2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

   


In [0]:
with tf.name_scope('loss'):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
  loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
with tf.name_scope('train'):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  trainining_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope('eval'):
  correct = tf.nn.in_top_k(logits, y, 1)
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
saver = tf.train.Saver()

In [0]:
n_epochs = 20
n_batches = 50


In [61]:
with tf.Session() as sess: 
  tf.global_variables_initializer().run() 

  for epoch in range(n_epochs): 
    for X_batch, y_batch in shuffle_batch(X_train, y_train, n_batches): 
      sess.run(trainining_op, feed_dict={X: X_batch, y: y_batch})
    acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
    acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

  save_path = saver.save(sess, "./model_final_FNN.ckpt")

0 Batch accuracy: 0.9 Validation accuracy: 0.9028
1 Batch accuracy: 0.92 Validation accuracy: 0.9254
2 Batch accuracy: 0.94 Validation accuracy: 0.9374
3 Batch accuracy: 0.9 Validation accuracy: 0.942
4 Batch accuracy: 0.94 Validation accuracy: 0.9472
5 Batch accuracy: 0.94 Validation accuracy: 0.9514
6 Batch accuracy: 1.0 Validation accuracy: 0.9548
7 Batch accuracy: 0.94 Validation accuracy: 0.9612
8 Batch accuracy: 0.96 Validation accuracy: 0.9618
9 Batch accuracy: 0.94 Validation accuracy: 0.965
10 Batch accuracy: 0.92 Validation accuracy: 0.9654
11 Batch accuracy: 0.98 Validation accuracy: 0.9668
12 Batch accuracy: 0.98 Validation accuracy: 0.9684
13 Batch accuracy: 0.98 Validation accuracy: 0.9702
14 Batch accuracy: 1.0 Validation accuracy: 0.9696
15 Batch accuracy: 0.94 Validation accuracy: 0.9714
16 Batch accuracy: 0.98 Validation accuracy: 0.9726
17 Batch accuracy: 1.0 Validation accuracy: 0.9728
18 Batch accuracy: 0.98 Validation accuracy: 0.975
19 Batch accuracy: 0.98 Valida

In [62]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
tf.compat.v1.disable_eager_execution()


In [0]:
from tensorflow.python.summary.writer.writer import FileWriter


In [66]:
FileWriter('logs/train').close()
%tensorboard --logdir logs/train

Reusing TensorBoard on port 6006 (pid 657), started 0:10:35 ago. (Use '!kill 657' to kill it.)